# SEIRHVD Data Fit

This Jupyter notebooks implements the SEIRHVD4 model for fitting the actual Chilean data in order to make projections in bed usage, amount of infected and deaths. 
This implements a single instance to facilitate understanding the model prior to run multiple data fittings at a time.

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime
from numpy import linalg as LA

import platform
OS = platform.system()

import matplotlib.pyplot as plt
if OS == 'Linux':    
    %matplotlib tk
    print('Linux')
elif OS == 'Windows':
    %matplotlib qt
    print('Windows')
else:
    print('OS not detected :-|')

import sys
from pathlib import Path
sys.path.insert(1, '../src/SEIRHVD/')
sys.path.insert(1, '../src/utils/')
sys.path.insert(1, 'src/SEIRHVD/')
sys.path.insert(1, 'src/utils/')

from class_SEIRHUVD4 import SEIRHVD 
from Quarantine import Quarantine
from importdata import ImportData

Linux


## Time Variables
Declare Initial date for simulating and data fitting

In [2]:
initdate = datetime(2020,5,15)
currentdate = datetime.now()
currentday = (currentdate - initdate).days

## Import Data
To import data we use an ImportData object which is initialized with the region's cut and the initial date. Meanwhile we will adjust this data for regions, then I'll add Heath sector as the minimum territorial unit for SEIRHVD models.

In [3]:
tstate = '13'

# Import Data
RM = ImportData(tstate=tstate,initdate = initdate)
RM.importdata()

Importing General Data
Importing Population
Importing Active infected
Importing Accumulated Infected
Importing Daily Infected
Importing Sochimi Data
Importing Accumulated Deaths
Importing Active Infected by Minciencia
Done


## Simulation Parameters

In [4]:
# Simulation parameters
tsim = 1000
beta = 0.28
mu = 0.8
k = 0.1

SeroPrevFactor=0.035 # SeroPrevalence Factor (The proportion of the population that participates in the dynamics)
expinfection=0 # Proportion in which the exposed infect - 0: nothing, 1: equally as Infected

## Quarantines
Build Quarantine Object:
 
     Q = Quarantine(rem_mob,max_mob=max_mob,qp=0,iqt=0,fqt=1000,movfunct = 'once')
     alpha = Q.alpha
     
     Plot Quarantine dynamics:
         Q.plot()

In [5]:
# Quarantines 
max_mob = 0.6
rem_mob = 0.4

#alpha = Quarantine(rem_mob,max_mob=max_mob,qp=0,iqt=0,fqt=1000,movfunct = 'once').alpha(t)
alpha = Quarantine(rem_mob).alpha

## Underreport
Fraction of Infected detected/reported  
    - Imi_det: fraction of Mild detected
    - Imi_as: fraction of asymptomatic detected

In [6]:
Imi_det = 0.56
Ias_det = 0

## Simulation
Initialize Simulation Object, and run the simulation. 

In [7]:
simulation = SEIRHVD(tsim,beta,mu,alpha,k=k,SeroPrevFactor=SeroPrevFactor,expinfection=expinfection,InitialConditions = RM,Imi_det = Imi_det,Ias_det = Ias_det)
simulation.integr_sci(0,tsim,0.1)

InitialCondition Object Data


  message: 'The solver successfully reached the end of the integration interval.'
     nfev: 2870
     njev: 51
      nlu: 51
      sol: None
   status: 0
  success: True
        t: array([0.00000000e+00, 1.07133033e-08, 2.14266065e-08, 1.07154459e-04,
       2.14287492e-04, 3.21420524e-04, 1.39275085e-03, 2.46408118e-03,
       3.53541150e-03, 4.60674183e-03, 1.53200451e-02, 2.60333484e-02,
       3.67466516e-02, 4.74599549e-02, 5.81732581e-02, 1.65306291e-01,
       2.72439323e-01, 3.79572356e-01, 4.86705389e-01, 5.93838421e-01,
       7.00971454e-01, 1.06493835e+00, 1.42890524e+00, 1.79287213e+00,
       1.88386385e+00, 1.97485558e+00, 1.99760351e+00, 2.02035144e+00,
       2.02148883e+00, 2.02262623e+00, 2.02376363e+00, 2.02510481e+00,
       2.02600119e+00, 2.02666929e+00, 2.02720112e+00, 2.02773294e+00,
       2.02810363e+00, 2.02847433e+00, 2.02877012e+00, 2.02906591e+00,
       2.02936170e+00, 2.02971665e+00, 2.03007160e+00, 2.03042656e+00,
       2.03078151e+00, 2.03120739e+00

# Analysis - Detected infected

The following plots will show the simulation results with the real data

In [13]:
# Days to plot (Today + ..)
days = currentday+20
index = np.searchsorted(simulation.t,days)

## Detected Active Infected
Here we define a Detected infected population as follows:  

 Idet = Imi*det + Ise + Icr  

where det is the proportion of mild detections


**Aun me falta corregir el valor inicial, pues este lo considera para todos los infectados**

In [14]:
tr_index = np.searchsorted(RM.tr,days)

plt.scatter(RM.tr,RM.Ir,label='Real Data')

plt.xlim(0,days)
plt.plot(simulation.t[:index],simulation.I_det[:index],label='Infected')
plt.legend(loc=0)
plt.title('Detectable Active Infected')
plt.show()


## Detected Daily Infected

In [11]:
plt.xlim(0,days)
plt.plot(simulation.t[:index],simulation.I_d_det[:index],label='Daily new Infected')
plt.scatter(RM.I_d_r_tr,RM.I_d_r,label='Real Data')
plt.legend(loc=0)
plt.title('Detected Daily infected')
plt.show()

## Detected Accumulated Infected

In [12]:
plt.xlim(0,days)
plt.plot(simulation.t[:index],simulation.I_ac_det[:index],label='Total Acummulated Infected')
plt.scatter(RM.I_ac_r_tr,RM.I_ac_r,label='Real Data')
plt.legend(loc=0)
plt.show()

# Analysis - Total infected

The following plots will show the simulation results with the real data

## Total Active Infected

In [ ]:
tr_index = np.searchsorted(RM.tr,days)

plt.scatter(RM.tr,RM.Ir,label='Real Data')

plt.xlim(0,days)
plt.plot(simulation.t[:index],simulation.I[:index],label='Total Active Infected')
plt.legend(loc=0)
plt.title('Total Active Infected')
plt.show()


## Daily Infected

In [ ]:
plt.xlim(0,days)
plt.plot(simulation.t[:index],simulation.I_d[:index],label='Total Daily new Infected')
plt.scatter(RM.I_d_r_tr,RM.I_d_r,label='Real Data')
plt.legend(loc=0)
plt.show()

## Acummulated Infected

In [ ]:
plt.xlim(0,days)
plt.plot(simulation.t[:index],simulation.I_ac[:index],label='Total Acummulated Infected')
plt.scatter(RM.I_ac_r_tr,RM.I_ac_r,label='Real Data')
plt.legend(loc=0)
plt.show()

## Deaths

In [ ]:
plt.xlim(0,days)
plt.plot(simulation.t[:index],simulation.B[:index],label='Total Acummulated Deaths')
plt.scatter(RM.Br_tr,RM.Br,label='Real Data')
plt.legend(loc=0)
plt.show()

## UCI/UTI

In [ ]:
plt.xlim(0,days)
plt.plot(simulation.t[:index],simulation.Hse[:index]+simulation.Hout[:index],label='UCI/UTI Beds')
plt.scatter(RM.sochimi_tr,RM.Hr,label='Real Data')
plt.scatter(RM.sochimi_tr,RM.Hr_tot,label='Real Data')
plt.legend(loc=0)
plt.show()

## VMI

In [ ]:
plt.xlim(0,days)
plt.plot(simulation.t[:index],simulation.V[:index],label='VMI Usage')
plt.scatter(RM.sochimi_tr,RM.Vr,label='Real Data')
plt.legend(loc=0)
plt.show()

# Objective Functions



In [ ]:
# Date until which we calculate the error (normally when the quarantine dynamic change)
t1_date = datetime(2020,8,15)


t1_day = (t1_date-initdate).days
t1_idx = np.where(np.array(RM.tr)>=t1_day)[0][0]
idx = np.searchsorted(simulation.t,RM.tr[:t1_idx])

## Error:

In [ ]:
Err = np.sum(abs(RM.Ir[:t1_idx]-simulation.I[idx]))/np.mean(RM.Ir[:t1_idx])
print(Err)


In [ ]:
simulation.peak_date

In [ ]:
RM.initdate